In [ ]:
using Pkg
using LinearAlgebra
# using Plots
using Distributions
using Random
using StateSpaceDynamics


const SSD = StateSpaceDynamics

StateSpaceDynamics

# Gaussian HMM

In [2]:
"""
Create an underlying GaussianHMM to generate data
"""

# Create Guassian Emission Models
output_dim = 2
μ = [0.0, 0.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(output_dim, μ, Σ)

μ = [2.0, 1.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(output_dim, μ, Σ)

# Create GaussianHMM
true_model = SSD.GaussianHMM(K=2, output_dim=2)
true_model.B[1] = emission_1
true_model.B[2] = emission_2
true_model.A = [0.9 0.1; 0.8 0.2]

# Sample from the model
n=10000
true_labels, data = SSD.sample(true_model, n=n)

# Fit a gaussian hmm to the data
test_model = SSD.GaussianHMM(K=2, output_dim=2)
test_model.A = [0.8 0.2; 0.05 0.95]
ll = SSD.fit!(test_model, data)

print(isapprox(test_model.B[1].μ, true_model.B[1].μ, atol=0.1) || isapprox(test_model.B[1].μ, true_model.B[2].μ, atol=0.1))
print(isapprox(test_model.B[2].μ, true_model.B[2].μ, atol=0.1) || isapprox(test_model.B[2].μ, true_model.B[1].μ, atol=0.1))

Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:18 ( 0.18  s/it)


truetrue

# Switching Gaussian Regression #

In [3]:
# Create Emission Models
emission_1 = GaussianRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([3, 2, 2, 3], :, 1))
emission_2 = GaussianRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([-4, -2, 3, 2], :, 1))

# Create Switching Regression Model
true_model = SwitchingGaussianRegression(K=2, input_dim=3, output_dim=1, include_intercept=true)

# Plug in the emission models
true_model.B[1] = emission_1
true_model.B[2] = emission_2

# Sample from the model
n = 20000
Φ = randn(3, n)
true_labels, data = SSD.sample(true_model, Φ, n=n)

# Try to fit a new model to the data
test_model = SSD.SwitchingGaussianRegression(K=2, input_dim=3, output_dim=1, include_intercept=true)
ll = SSD.fit!(test_model, data, Φ)

# Test output -> not quite right yet
print(isapprox(test_model.B[1].β, true_model.B[1].β, atol=0.1) || isapprox(test_model.B[1].β, true_model.B[2].β, atol=0.1))
print(isapprox(test_model.B[2].β, true_model.B[2].β, atol=0.1) || isapprox(test_model.B[2].β, true_model.B[1].β, atol=0.1))

Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:24 ( 0.25  s/it)


truetrue

# Switching Bernoulli Regression #

In [6]:
"""
Create Bernoulli Regression Model
"""
# Make Emission Models
emission_1 = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([3, 1, 2], :, 1))
emission_2 = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-3, -2, 0.1], :, 1))

# Create Switching Bernoulli Regression and add the emissions
true_model = SSD.SwitchingBernoulliRegression(K=2, input_dim=2)
true_model.A = [0.9 0.1; 0.2 0.8]
true_model.B[1] = emission_1
true_model.B[2] = emission_2

# Sample from the model
n=40000
Φ = randn(2, n)
true_labels, data = SSD.sample(true_model, Φ, n=n)

# Fit a new Bernoulli Regression Model to the data
test_model = SSD.SwitchingBernoulliRegression(K=2, input_dim=2, λ=1.0)
test_model.A = [0.75 0.25; 0.1 0.9]
test_model.B[1] = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([2.5, 0.25, 1.0], :, 1))
test_model.B[2] = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-2.0, -3.0, -1.0], :, 1))
ll = SSD.fit!(test_model, data, Φ, max_iters=200)

# # Test it works alright
print(isapprox(test_model.B[1].β, true_model.B[1].β, atol=0.2) || isapprox(test_model.B[1].β, true_model.B[2].β, atol=0.2))
print(isapprox(test_model.B[2].β, true_model.B[2].β, atol=0.2) || isapprox(test_model.B[2].β, true_model.B[1].β, atol=0.2))

Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:01:45 ( 0.53  s/it)


truetrue

In [9]:
 # Make Emission Models
 emission_1 = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([3, 1, 2], :, 1))
 emission_2 = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-3, -2, 0.1], :, 1))

 # Create Switching Bernoulli Regression and add the emissions
 true_model = StateSpaceDynamics.SwitchingBernoulliRegression(K=2, input_dim=2)
 true_model.A = [0.9 0.1; 0.2 0.8]
 true_model.B[1] = emission_1
 true_model.B[2] = emission_2

 # Sample from the model
 n=20000
 Φ = randn(2, n)
 true_labels, data = StateSpaceDynamics.sample(true_model, Φ, n=n)

 # Fit a new Bernoulli Regression Model to the data
 test_model = StateSpaceDynamics.SwitchingBernoulliRegression(K=2, input_dim=2, λ=1.0)
 test_model.A = [0.75 0.25; 0.1 0.9]
 test_model.B[1] = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([2.5, 0.25, 1.0], :, 1))
 test_model.B[2] = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-2.0, -3.0, -1.0], :, 1))
 ll = StateSpaceDynamics.fit!(test_model, data, Φ, max_iters=200)


Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:49 ( 0.25  s/it)


34-element Vector{Float64}:
    -Inf
 -12999.549711873935
 -10283.761323822177
 -10182.552576531894
 -10163.157497212167
 -10157.805164091475
 -10155.77386284353
 -10154.79874771744
 -10154.27340076818
 -10153.976910824515
      ⋮
 -10153.573519534973
 -10153.573490853676
 -10153.573474870735
 -10153.573464835481
 -10153.573459206586
 -10153.573455784823
 -10153.573453857767
 -10153.57345270621
 -10153.5734520551

# Switching Poisson Regression

In [10]:
"""
Create a Switching Poisson Regression
"""
# Create the emission models
emission_1 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([4, 3, 2, 4], :, 1))
emission_2 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([-4, -2, 1, 3], :, 1))

# Initialize the SwitchingPoissonRegression
true_model = SwitchingPoissonRegression(K=2, input_dim=3, output_dim=1)

# Plug in the emission models
true_model.B[1] = emission_1
true_model.B[2] = emission_2

# Sample from the HMM
n=20000
Φ = randn(3, n)
true_labels, data = SSD.sample(true_model, Φ, n=n)

# Create a new SwitchingPoissonRegression and try to recover parameters
test_model = SwitchingPoissonRegression(K=2, input_dim=3, output_dim=1)

# Create the emission models for warm start
emission_1 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([2.0, 1.0, 4.0, 2.0], :, 1))
emission_2 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([-5.0, -1.0, 0.0, 2.0], :, 1))
test_model.B[1], test_model.B[2] = emission_1, emission_2

ll = SSD.fit!(test_model, data, Φ, max_iters=200)

print(isapprox(test_model.B[1].β, true_model.B[1].β, atol=0.1) || isapprox(test_model.B[1].β, true_model.B[2].β, atol=0.1))
print(isapprox(test_model.B[2].β, true_model.B[2].β, atol=0.1) || isapprox(test_model.B[2].β, true_model.B[1].β, atol=0.1))


Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:23 ( 0.12  s/it)


truetrue

In [ ]:
# Create Guassian Emission Models
output_dim = 2
μ = [-5.0, -4.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(output_dim=output_dim, μ=μ, Σ=Σ)

μ = [2.0, 1.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(output_dim=output_dim, μ=μ, Σ=Σ)

# Create GaussianHMM
true_model = GaussianHMM(K=2, output_dim=2)
true_model.B[1] = emission_1
true_model.B[2] = emission_2
true_model.A = [0.9 0.1; 0.8 0.2]

# Generate trialized synthetic data
n = 100
num_trials = 30
Y = Vector{Matrix{Float64}}(undef, num_trials)
trial_labels = Vector{Vector{Int}}(undef, num_trials)  

for i in 1:num_trials
    true_labels, data = StateSpaceDynamics.sample(true_model, n=n)  # Generate data and labels
    Y[i] = data  # Store data matrix for the ith trial
end

# Fit a model to the trialized synthetic data
est_model = GaussianHMM(K=2, output_dim=2)
lls = StateSpaceDynamics.fit!(est_model, Y, max_iters=100)

Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:21 ( 0.22  s/it)


LoadError: LoadError: UndefVarError: `@test` not defined
in expression starting at c:\Users\ryansenne\Documents\GitHub\ssm_julia\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X13sZmlsZQ==.jl:34

In [4]:
est_model.A

2×2 Matrix{Float64}:
 86.0          6.0
  3.77397e107  6.28994e106